<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Wine Reviews: Task 1
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Name: Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loanding and Exploration

In [7]:
data = pd.read_csv("winemag-data-130k-v2.csv")

# Remove non-usefule columns
data = data.drop(['taster_name', 'taster_twitter_handle'], axis=1)

In [15]:
print(data.dtypes)
print(data.shape)

Unnamed: 0       int64
country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
title           object
variety         object
winery          object
dtype: object
(129971, 12)


In [48]:
data_y = data['points']
non_text_data_x = data.drop(['description', 'points'], axis=1)
text_data_x = data[['description']]

In [49]:
print(data_y.shape)
print(non_text_data_x.shape)
print(text_data_x.shape)

(129971,)
(129971, 10)
(129971, 1)


## 1.1: Baseline model using non-text features

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [52]:
X_trainval, X_test, y_trainval, y_test = train_test_split(non_text_data_x, data_y)
continuous_features = ['price']
categorical_features = ['country', 'title', 'designation', 'province', 'region_1', 'region_2',
                        'variety', 'winery']

In [53]:
base_categorical_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = 'constant', fill_value = "missing")),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

base_continuous_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = "median")),
                                         ('scaler', StandardScaler())])

base_continuous_and_categorical_preprocessor = ColumnTransformer(
    transformers=[
        ('num', base_continuous_transformer, continuous_features),
        ('cat', base_categorical_transformer, categorical_features)
    ])

In [54]:
# Baseline Ridge
base_ridge_clf = Pipeline(steps=[('preprocessor', base_continuous_and_categorical_preprocessor),
                                ('classifier', Ridge())])

base_ridge_scores = cross_val_score(base_ridge_clf, X_trainval, y_trainval, cv=10)

print("Base Ridge Model with only cross val mean score: ", np.mean(base_ridge_scores))

Base Ridge Model with only cross val mean score:  0.5150964907714015


In [55]:
ridge_pipeline = Pipeline(steps=[('preprocessor', base_continuous_and_categorical_preprocessor),
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_trainval, y_trainval)

print(("Basic ridge score with gridsearch %.2f"
       %grid.score(X_test, y_test)))
print("Best param:", grid.best_params_)

Basic ridge score with gridsearch 0.52
Best param: {'regressor__alpha': 1}


## 1.2: Text-based model based on:
- Bag-of-Words
- Linear Model

In [68]:
from sklearn.feature_extraction.text import CountVectorizer

X_trainval_text, X_test_text, y_trainval_text, y_test_text = train_test_split(text_data_x, data_y)

In [70]:
vect = CountVectorizer(stop_words="english", max_features=1000)

X_train_transform = vect.fit_transform(X_trainval_text['description'])
X_test_text_transformed = vect.transform(X_test_text['description'])

In [71]:
print(X_train_transform.shape)
print(X_test_text_transformed.shape)

(97478, 1000)
(32493, 1000)


In [60]:
text_train

,description
58572,With its slightly sweet notes of pineapple pie...
99769,This perfumed wine has remarkable density from...
127329,"Made from a predominance of Pinot Noir, with 8..."
73570,"There's a thick, slightly muddled quality to t..."
74965,Intriguing hints of passion fruit and tomato l...
...,...
91314,"Made of 90% Sangiovese, 5% Colorino and 5% Mer..."
45395,"A blend of 70% Sangiovese, 20% Merlot and 10% ..."
43288,"With slightly sharp, herbal red-fruit aromas, ..."
102543,A touch of honey brings a sense of sweetness t...


In [73]:
ridge_pipeline = Pipeline(steps=[('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_train_transform, y_trainval_text)

print(("Basic ridge score with gridsearch %.2f"
       %grid.score(X_test_text_transformed, y_test_text)))
print("Best param:", grid.best_params_)

Basic ridge score with gridsearch 0.61
Best param: {'regressor__alpha': 10}


## 1.3: More text-based models:
- n-grams
- characters
- tf-idf
- other ways to tune BoW

## 1.4: Text and non-text features hybrid model